In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from transformers import BertTokenizer, BertModel
# from employees_dummy_data import data_elastic

In [3]:
try:
    es = Elasticsearch(cloud_id='69308a62925f4983ad0027b5a4e54a37:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDc0OTZlN2NmYjAyNzRjYmVhMmI2ZGYwYjM0N2EwZWE2JDBjZTgzY2YwMDg5MDRjYzZiMjZkZjcyNmFmZjIxMmQy', 
                       api_key="UTI4aEtKVUJXY0otSVhDektkQm46U2dSQmFJLVJROFNSbHRDRnVKMTI4QQ==")

    if es.ping():
        print("Connected to Elasticsearch")
    else:
        print("Elasticsearch connection failed")
except es_exceptions.ConnectionError as e:
    print(f"Elasticsearch Connection Error: {e}")


Connected to Elasticsearch


In [ ]:
# Initialize BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def preprocess_and_index(data):    
    for profile in data:
        doc_id = profile['_id']
        # Combine text fields for embedding
        text = ' '.join(profile['technical_skills'] + profile['industry_skills'] + [profile['bio']])
        
        # Tokenise and get embeddings
        inputs = tokenizer(text, return_tensors='pt')
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
        
        # Index the profile in Elasticsearch
        try:
            es.index(index='profiles', id=doc_id, body={
                'name': profile['name'],
                'email': profile['email'],
                'technical_skills': profile['technical_skills'],
                'industry_skills': profile['industry_skills'],
                'bio': profile['bio'],
                'embedding': embedding.tolist()
            })
            print(f"Indexed profile {doc_id} successfully")
        except es_exceptions.ConnectionError as e:
            print(f"Elasticsearch Indexing Error: {e}")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
def search_profiles(query):
    # Preprocess query
    inputs = tokenizer(query, return_tensors='pt')
    outputs = model(**inputs)
    query_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    
    # Search in Elasticsearch
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                "params": {"query_vector": query_embedding.tolist()}
            }
        }
    }

    try:
        response = es.search(index='profiles', body={"query": script_query})
        return response['hits']['hits']
    except es_exceptions.ConnectionError as e:
        print(f"Elasticsearch Search Error: {e}")
        return []

In [7]:
query = "Java"

preprocess_and_index(data_elastic)

results = search_profiles(query)

for result in results:
        print(result['_source']['name'], 
              result['_source']['email'], 
              result['_score'],
              result['_source']['technical_skills']
              )



Diana Evans diana@example.com 1.4669056 ['Python', 'Machine Learning', 'TensorFlow']
John Doe john@example.com 1.4549011 ['Python', 'JavaScript']
Charlie Davis charlie@example.com 1.4401659 ['PHP', 'Laravel', 'MySQL']
Hannah White hannah@example.com 1.4116818 ['HTML', 'CSS', 'JavaScript', 'Vue.js']
Bob Brown bob@example.com 1.4087967 ['Ruby', 'Rails', 'JavaScript']
Ethan Foster ethan@example.com 1.4074528 ['JavaScript', 'React', 'Node.js']
Alice Johnson alice@example.com 1.3966638 ['C++', 'C#', '.NET']
Fiona Green fiona@example.com 1.3912638 ['Swift', 'Objective-C', 'iOS Development']
George Harris george@example.com 1.3806974 ['Go', 'Kubernetes', 'Docker']
Jane Smith jane@example.com 1.3640718 ['Java', 'Spring', 'Hibernate']
